In [1]:
from __future__ import division
import pandas as pd
import numpy


In [2]:
df1_WS1T = pd.read_csv("/Users/asteriag/Desktop/DATASCIENCEPROJECT_WORKFLOW/TimeStamp_Engineering/engineering_OF_segment1_workflowS1_has_duration.csv",engine='python')

In [3]:
df1_WS1T.head(n=1)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,ITEM_CRUSER,WRKFLW_ORG_TYPE_CRUSER,STATUS_ORDER,LATEST_STATUS,WRKFLW_PROCESS,QUERY_PART,...,SPK_CAT_TYPE_CD,BROAD_COURSE_LEVEL,DEGREETYPE,LIABILITY_CATEGORY_CD_RECODE,ORIG_LIABILITY_CATEGORY_CD_RECODE,OFFER_RELEASED_FG,OFFER_OUTCOME_CD,OFFER_CONDITIONS,time_used_str,time_used_second
0,1126888|499828|767443,Submit,Submit,17/7/2019 1:01:31,PADWYER,Academic Services,24,N,1. App to Assessment,Part 1 - Application ID Workflow,...,MCW,HDCW,Full Degree,OF,OF,Y,NaN,Y,29 days 12 hours 51 minutes 40 seconds,2551900.0


In [4]:
numOfWorkflowProcesses = df1_WS1T.groupby('CASE_ID').CASE_ID.count()

count_dic=dict()
for i,v in numOfWorkflowProcesses.items():
    if count_dic.get(v) is None:
        count_dic[v]=[]
        count_dic[v].append(i)
    else:
        count_dic[v].append(i)
#count_dic
count_dic = dict(sorted(count_dic.items()))

In [5]:
len_list=[]
for key in count_dic.keys():
    len_list.append(len(count_dic[key]))

In [6]:
def strfdelta(tdelta, fmt):
    d=dict()
    d["days"],rem1 = divmod(tdelta,60*60*24)
    d["hours"], rem2 = divmod(rem1, 3600)
    d["minutes"], d["seconds"] = divmod(rem2, 60)
    return fmt.format(**d)

In [7]:
list_avgtime_per_key =[]
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_person = 0
    for id in id_list:
        df = df1_WS1T[df1_WS1T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_person+=tsum
    list_avgtime_per_key.append(int((list_sumtime_person/len(count_dic[key]))))

list_avgtime_per_key_str = []
for i in list_avgtime_per_key:
    list_avgtime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [8]:
list_mediantime_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_per_person = []
    for id in id_list:
        df = df1_WS1T[df1_WS1T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_per_person.append(tsum)
    list_mediantime_per_key.append(numpy.median(list_sumtime_per_person))

list_mediantime_per_key_str = []
for i in list_mediantime_per_key:
    list_mediantime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [9]:
list_percent_reach_end = []
for key in count_dic.keys():
    id_list = count_dic[key]
    count_df_end_step = 0
    count_df_id = 0
    for id in id_list:
        df_id = df1_WS1T[df1_WS1T['CASE_ID']== id]
        df_end_step = df_id[df_id['WRKFLW_TSK_MAP_GROUPED']=='Send for Assessment']
        count_df_id += df_id['CASE_ID'].nunique()
        count_df_end_step += df_end_step['CASE_ID'].nunique()
    list_percent_reach_end.append(str(round((count_df_end_step/count_df_id)*100,2))+ "%")

In [10]:
list_step_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = set()
    for id in id_list:
        df = df1_WS1T[df1_WS1T['CASE_ID']== id]
        for index, row in df.iterrows():
            list_step.add(row['WRKFLW_TSK_MAP_GROUPED'])
    list_step_per_key.append(list_step)


In [11]:
from collections import Counter
import collections, functools, operator
list_step_sum_time_per_key = []
list_step_count_per_key = []
list_step_ave_time_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = []
    step_time_list = []
    for id in id_list:
        df = df1_WS1T[df1_WS1T['CASE_ID']== id]
        step_time = dict(zip(df.WRKFLW_TSK_MAP_GROUPED, df.time_used_second))
        step_time_list.append(step_time)
        for index, row in df.iterrows():
            list_step.append(row['WRKFLW_TSK_MAP_GROUPED'])

    step_sum_time = dict(functools.reduce(operator.add,
         map(collections.Counter, step_time_list)))

    dict_list_step=dict(Counter(list_step))

    step_ave_time = {key: strfdelta(step_sum_time[key] // dict_list_step.get(key, 0),"{days} days {hours} hours {minutes} minutes {seconds} seconds")
                        for key in step_sum_time.keys()}

    sort_step_sum_time = sorted(step_sum_time.items(),key = lambda x: x[1], reverse = True)
    list_step_sum_time_per_key.append(sort_step_sum_time)

    sort_dict_list_step = sorted(dict_list_step.items(),key = lambda x: x[1], reverse = True)
    list_step_count_per_key.append(sort_dict_list_step)

    sort_step_ave_time = sorted(step_ave_time.items(),key = lambda x: x[1], reverse = True)
    list_step_ave_time_per_key.append(sort_step_ave_time)

In [12]:
s1_t1=pd.DataFrame({'Number of Workflow processes in this Workflow Segment':count_dic.keys(),'Number of cases':len_list, 'average time for workflow segment':list_avgtime_per_key_str,'median time for workflow segment':list_mediantime_per_key_str,'percentage of cases reaching to end of workflow segment':list_percent_reach_end,'step':list_step_per_key,'step count':list_step_count_per_key})
s1_t1['Num of Occurrences per step(descending)'] = list_step_count_per_key
s1_t1['Average time spends per step(descending)'] = list_step_ave_time_per_key
s1_t1.head(n=10)


,Number of Workflow processes in this Workflow Segment,Number of cases,average time for workflow segment,median time for workflow segment,percentage of cases reaching to end of workflow segment,step,step count,Num of Occurrences per step(descending),Average time spends per step(descending)
0,1,537,0 days 0 hours 0 minutes 0 seconds,0.0 days 0.0 hours 0.0 minutes 0.0 seconds,0.0%,{Submit},"[(Submit, 537)]","[(Submit, 537)]",[]
1,2,177,17 days 17 hours 53 minutes 27 seconds,9.0 days 22.0 hours 36.0 minutes 34.0 seconds,76.84%,"{WITHDRAW & LOCK, AR - Reviewed - previous or ...","[(Submit, 177), (Send for Assessment, 136), (W...","[(Submit, 177), (Send for Assessment, 136), (W...","[(AR - Created - applicant demographics, 4.0 d..."
2,3,178,16 days 10 hours 55 minutes 13 seconds,9.0 days 13.0 hours 35.0 minutes 11.0 seconds,74.72%,"{AR - Created - Other, AR - Created - previous...","[(Submit, 181), (Send for Assessment, 133), (A...","[(Submit, 181), (Send for Assessment, 133), (A...","[(Put Triage On-Hold, 5.0 days 13.0 hours 5.0 ..."
3,4,550,15 days 13 hours 3 minutes 14 seconds,10.0 days 21.0 hours 40.0 minutes 55.5 seconds,90.55%,{AR - Not Satisfied - previous or current stud...,"[(Submit, 553), (Send for Assessment, 498), (A...","[(Submit, 553), (Send for Assessment, 498), (A...","[(Put Triage On-Hold, 5.0 days 4.0 hours 54.0 ..."
4,5,928,16 days 7 hours 45 minutes 46 seconds,11.0 days 15.0 hours 24.0 minutes 20.5 seconds,95.15%,{AR - Not Satisfied - previous or current stud...,"[(Submit, 933), (Send for Assessment, 883), (A...","[(Submit, 933), (Send for Assessment, 883), (A...","[(AR - Created - further information, 8.0 days..."
5,6,1132,20 days 5 hours 0 minutes 43 seconds,13.0 days 5.0 hours 54.0 minutes 39.5 seconds,93.37%,{AR - Not Satisfied - previous or current stud...,"[(Submit, 1135), (Send for Assessment, 1057), ...","[(Submit, 1135), (Send for Assessment, 1057), ...","[(AR - Requested - previous or current study, ..."
6,7,1470,20 days 7 hours 38 minutes 30 seconds,13.0 days 23.0 hours 35.0 minutes 49.5 seconds,93.06%,{AR - Not Satisfied - previous or current stud...,"[(Submit, 1477), (AR - Satisfied - previous or...","[(Submit, 1477), (AR - Satisfied - previous or...","[(AR - Requested - previous or current study, ..."
7,8,1479,21 days 12 hours 38 minutes 35 seconds,14.0 days 23.0 hours 33.0 minutes 27.0 seconds,92.43%,{AR - Not Satisfied - previous or current stud...,"[(AR - Satisfied - previous or current study, ...","[(AR - Satisfied - previous or current study, ...","[(AR - Created - applicant demographics, 5.0 d..."
8,9,1329,24 days 22 hours 58 minutes 2 seconds,17.0 days 11.0 hours 3.0 minutes 38.0 seconds,93.08%,{AR - Not Satisfied - previous or current stud...,"[(AR - Satisfied - previous or current study, ...","[(AR - Satisfied - previous or current study, ...","[(AR - Satisfied - applicant demographics, 7.0..."
9,10,1116,27 days 13 hours 50 minutes 44 seconds,19.0 days 16.0 hours 53.0 minutes 45.5 seconds,91.4%,{AR - Not Satisfied - previous or current stud...,"[(Submit, 1120), (AR - Satisfied - previous or...","[(Submit, 1120), (AR - Satisfied - previous or...","[(AR - Created - syllabus, 6.0 days 23.0 hours..."


In [13]:
s1_t1.to_csv('engineering_OF_segment1_workflowS1_table.csv', index=False)






